In [1]:
import sys
sys.path.append('..')
import ast
import scipy.io
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import statistics
import pandas as pd
import pickle
import os
from constant import path, parameter
from lib import evaluate

In [2]:
import warnings
warnings.filterwarnings('ignore')

# KNN,Naive Bayes,LogisticRegression algorithms

In [3]:
def training_models(models,features_train,targets_train,features_test):
    meta_proba = np.zeros((len(models) * 2, features_test.shape[0]))
    for i in range(len(models)):
        learner = models[i]
        learner.fit(features_train,targets_train)
        predictions_proba = learner.predict_proba(features_test)
        meta_proba[2*i][:] = predictions_proba.T[0]
        meta_proba[2*i + 1][:] = predictions_proba.T[1]
    meta_proba = meta_proba.transpose()
    return meta_proba

In [4]:
path.MODEL_PATH

'../model'

In [5]:
models = parameter.MODEL
models

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 GaussianNB(priors=None, var_smoothing=1e-09),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                      weights='uniform')]

In [6]:
models[0].fit([[1,2,3],[2,1,4],[1,1,0]],[1,2,3])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
from train import TrainMaster
train_master = TrainMaster()
for s in models:
    train_master.training_phase(model = s,features_train=[[1,2,3],[2,1,4],[1,1,0]],targets_train=[1,2,3])

----- Dump model LogisticRegression to success! -----
----- Dump model GaussianNB to success! -----
----- Dump model KNeighborsClassifier to success! -----


In [8]:
x = np.random.rand(10,6).reshape(10, 6)
# matrix = np.hsplit(x, 3)
# matrix
# combining = np.zeros((10,2))
# for i in range(3):
#     combining += matrix[i] 
# print(combining)
print(x)
rules = evaluate.Rules()
print(rules.combining_sum_rule(x, 2))
print(rules.combining_product_rule(x, 2))
print(rules.combining_max_min(x, 2, parameter ='max'))
min_value = rules.combining_max_min(x, 2, parameter ='min')

[[0.54152175 0.39872895 0.33780843 0.60394648 0.35850239 0.63073175]
 [0.6825092  0.8937595  0.87214547 0.220398   0.04268019 0.25928691]
 [0.61262267 0.23622209 0.94254647 0.4692196  0.42170965 0.41523484]
 [0.71619133 0.11879307 0.15935087 0.46141659 0.68019061 0.08568724]
 [0.0081002  0.38179384 0.87741736 0.08093596 0.71737675 0.16442789]
 [0.90750098 0.88167341 0.41186004 0.80517205 0.73014711 0.29000414]
 [0.99973194 0.96640253 0.14818585 0.70166128 0.09056392 0.87511675]
 [0.5476114  0.19811232 0.11478263 0.28411075 0.86699143 0.99418481]
 [0.07206965 0.81323832 0.28787329 0.35879684 0.40013193 0.05612491]
 [0.56715802 0.91617505 0.62558767 0.56440021 0.71183687 0.85642174]]
[[0.38182274 0.25241045 0.32284673]
 [0.30096907 0.31214656 0.37714413]
 [0.36061409 0.21931058 0.45259377]
 [0.39253597 0.26632789 0.08167937]
 [0.02967872 0.36639019 0.34728175]
 [0.57089101 0.5372735  0.23395473]
 [0.56713107 0.35232215 0.34110087]
 [0.27724072 0.35503459 0.36965581]
 [0.14362216 0.404456

In [9]:
targets = np.unique([1,2,3,1,2,2])

In [10]:
import pandas as pd
df = pd.DataFrame(min_value,columns=targets)
print(df)
a = df.idxmax(axis=1)
a
for i in a :
    print(i)

          1         2         3
0  0.541522  0.358502  0.337808
1  0.220398  0.042680  0.259287
2  0.469220  0.236222  0.415235
3  0.461417  0.118793  0.085687
4  0.008100  0.381794  0.164428
5  0.805172  0.730147  0.290004
6  0.701661  0.090564  0.148186
7  0.284111  0.198112  0.114783
8  0.072070  0.400132  0.056125
9  0.564400  0.711837  0.625588
1
3
1
1
2
1
1
1
2
2


In [18]:
a = rules.target(min_value, targets)
def error_combining_rule(target_combining_rule, target_test):
    boolen_result = []
    for i in range(len(target_combining_rule)):
        result = 1 if target_combining_rule[i] != target_test[i] else 0
        boolen_result.append(result)
    mean_combining_rule = statistics.mean(boolen_result)
#     variance_combining_rule = statistics.variance(boolen_result)
    return mean_combining_rule
error_combining_rule(a,[1,2,1,1,2,1,1,1,2,3])

0.2

# Caculate Mean and Variance

In [12]:
def combining_sum_product(matrix1, matrix2, matrix3):
    #Combining Algorithms use Sum Rules
    combining_sum_rule = matrix1 + matrix2 + matrix3
    combining_sum_rule = combining_sum_rule * 1/3   #Mean elements in matrix
    #Combining algorithms PRODUCT RULES
    combining_product_rule =  matrix1 * matrix2 * matrix3 * 1/3
    return combining_sum_rule,combining_product_rule

In [13]:
#Combining algorithms MIN RULES
#Function compare element row in matrix => Min value 
def min_element(value1, value2, value3):
    min_value = min([value1, value2, value3])
    return min_value 

def row_in_matrix(i,matrix_1, matrix_2, matrix_3):
    min_value_class1 = min_element(value1=matrix_1[i][0], value2=matrix_2[i][0], value3= matrix_3[i][0])
    min_value_class2 = min_element(value1=matrix_1[i][1], value2=matrix_2[i][1], value3= matrix_3[i][1])
    return min_value_class1, min_value_class2
    
def min_rule(matrix_1, matrix_2, matrix_3):
    combining_min_rule = np.zeros((matrix_1.shape))
    
    #Store variables to combining matrix
    for i in range(len(matrix_1)):
        combining_min_rule[i] = row_in_matrix(i, matrix_1=matrix_1,
                                         matrix_2=matrix_2,matrix_3=matrix_3)
    return combining_min_rule


In [14]:
#Combining algorithms MAX RULES
#Create variables to store combining matrix
def max_element(value1, value2, value3):
        max_value = max([value1, value2, value3])
        return max_value 
#Function compare element row in matrix => Max value 
def row_in_matrix(i,matrix_1, matrix_2, matrix_3):
        max_value_class1 = max_element(value1=matrix_1[i][0], value2=matrix_2[i][0], value3= matrix_3[i][0])
        max_value_class2 = max_element(value1=matrix_1[i][1], value2=matrix_2[i][1], value3= matrix_3[i][1])
        return max_value_class1, max_value_class2
    
def max_rule(matrix_1, matrix_2, matrix_3):
    combining_max_rule = np.zeros((matrix_1.shape))
    
    #Store variables to combining matrix
    for i in range(len(matrix_1)):
        combining_max_rule[i] = row_in_matrix(i, matrix_1=matrix_1,
                                             matrix_2=matrix_2,matrix_3=matrix_3)
    return combining_max_rule


In [15]:
def target(combining_matrix):
    targets_combining_algorithm = []
    for row in combining_matrix:
        result = 1 if row[0] > row[1] else 2
        targets_combining_algorithm.append(result)
    targets_combining_algorithm = np.asarray(targets_combining_algorithm)
    return targets_combining_algorithm
    
def error_combining_rule(target_combining_rule, target_test):
    boolen_result = []
    for i in range(len(target_test)):
        result = 1 if target_combining_rule[i] != target_test[i] else 0
        boolen_result.append(result)
    mean_combining_rule = statistics.mean(boolen_result)
#     variance_combining_rule = statistics.variance(boolen_result)
    return mean_combining_rule

In [16]:
def process_data_dat(filename_dat):
    samples = []
    data_set = [i.strip().split() for i in open("../data/" + filename_dat).readlines()] 
    for sample in data_set:
        res = ast.literal_eval(sample[0])
        sample = list(res)
        samples.append(sample)
    samples = np.asarray(samples)
    return samples

def process_cv_filename(cv_file):
    cv_mat = scipy.io.loadmat('../data/' + cv_file)
    return cv_mat['cv']

def split_train_test_by_id(data,cv_file, niters, nfolds):
    arrErrSum = []
    arrErrProd = []
    arrErrMax= []
    arrErrMin = []
    for i in range(niters):
        for j in range(nfolds):
            train_index = []
            cv_test = process_cv_filename(cv_file)
            test_index = cv_test[0][i*nfolds + j]
            test_index = np.concatenate(([i-1 for i in test_index]))
#             print("LOOP:",i*nfolds + j)
            train_index.append([i for i in range(len(data)) if i not in test_index]) 
            train_index = np.asarray(train_index[0])
#             print(data[train_index])
#             print(data[test_index])
            meta_proba = training_models(models,features_train=data[train_index][:,0:data.shape[1] - 1],
                               targets_train=data[train_index][:,-1], features_test=data[test_index][:,0:data.shape[1] - 1])
            predict_lr_proba = meta_proba[:,0:2]
            predict_gnb_proba = meta_proba[:,2:4]
            predict_knn_proba = meta_proba[:,4:6]
            #Caculate Mean and Variance by Sum Rules:
            combining_sum_rule,combining_product_rule = combining_sum_product(predict_lr_proba,predict_gnb_proba,
                                                                             predict_knn_proba)
            targets_combining_sum_rule = target(combining_sum_rule)
            mean_combining_sum_rule = error_combining_rule(targets_combining_sum_rule, data[test_index][:,-1] )
            arrErrSum.append(mean_combining_sum_rule)
            
            #Caculate Mean and Variance by Product Rules:
            targets_combining_product_rule = target(combining_product_rule)
            mean_combining_product_rule = error_combining_rule(targets_combining_product_rule, data[test_index][:,-1] )
            arrErrProd.append(mean_combining_product_rule)
            
            #Caculate Mean and Variance by Min Rules:
            combining_min_rule = min_rule(predict_lr_proba,predict_gnb_proba,predict_knn_proba)
            targets_combining_min_rule = target(combining_min_rule)
            mean_combining_min_rule = error_combining_rule(targets_combining_min_rule, data[test_index][:,-1] )
            arrErrMin.append(mean_combining_min_rule)
            
            #Caculate Mean and Variance by Max Rules:
            combining_max_rule = max_rule(predict_lr_proba,predict_gnb_proba,predict_knn_proba)
            targets_combining_max_rule = target(combining_max_rule)
            mean_combining_max_rule = error_combining_rule(targets_combining_max_rule, data[test_index][:,-1] )
            arrErrMax.append(mean_combining_max_rule)
#     #Caculate mean and variance Total Sum Rule
#     mean_sum = statistics.mean(arrErrSum)
#     variance_sum = statistics.variance(arrErrSum)
#     #Caculate mean and variance Total Product Rule
#     mean_product = statistics.mean(arrErrProd)
#     variance_product = statistics.variance(arrErrProd)
#     #Caculate mean and variance Total Min Rule
#     mean_min = statistics.mean(arrErrMin)
#     variance_min = statistics.variance(arrErrMin)
#     #Caculate mean and variance Total Max Rule
#     mean_max = statistics.mean(arrErrMax)
#     variance_max = statistics.variance(arrErrMax)
    
    pickle_file = {'Dataset':cv_file,'arrErrSum':arrErrSum,'arrErrProd':arrErrProd,
                   'arrErrMin':arrErrMin,'arrErrMax':arrErrMax}
#     pickle_file = {'Dataset':cv_file,'mean_sum':mean_sum,'variance_sum':variance_sum,
#                    'mean_product':mean_product,'variance_product':variance_product,
#                   'mean_min':mean_min,'variance_min':variance_min,
#                   'mean_max':mean_max,'variance_max':variance_max}
    print(pickle_file)
    pickle_out = open("dict_{}.pickle".format(cv_file),"wb")
    pickle.dump(pickle_file, pickle_out)
    pickle_out.close()
    
    

In [17]:
dataframe = []
for i in range(len(data_dat)):
    data = process_data_dat(data_dat[i])
    split_train_test_by_id(data,cv_filename[i],niters=3,nfolds=10)
    pickle_in = open("dict_{}.pickle".format(cv_filename[i]),"rb")
    example_dict = pickle.load(pickle_in)
    dataframe.append(example_dict)

NameError: name 'data_dat' is not defined

In [ ]:
import pandas as pd
data = pd.DataFrame.from_dict(dataframe)
data

In [ ]:
def mean_rules(cv_file,arrErrSum,arrErrProd,arrErrMin,arrErrMax):
    #Caculate mean and variance Total Sum Rule
    mean_sum = statistics.mean(arrErrSum)
    variance_sum = statistics.variance(arrErrSum)
    #Caculate mean and variance Total Product Rule
    mean_product = statistics.mean(arrErrProd)
    variance_product = statistics.variance(arrErrProd)
    #Caculate mean and variance Total Min Rule
    mean_min = statistics.mean(arrErrMin)
    variance_min = statistics.variance(arrErrMin)
    #Caculate mean and variance Total Max Rule
    mean_max = statistics.mean(arrErrMax)
    variance_max = statistics.variance(arrErrMax)
    dict_data = {'Dataset':cv_file,'mean_sum':mean_sum,'variance_sum':variance_sum,
                   'mean_product':mean_product,'variance_product':variance_product,
                  'mean_min':mean_min,'variance_min':variance_min,
                  'mean_max':mean_max,'variance_max':variance_max}
    return dict_data

In [ ]:
dict_data = []
for i in range(len(cv_filename)):
    dict_data.append(mean_rules(cv_filename[i],data['arrErrSum'][i],data['arrErrProd'][i],
                               data['arrErrMin'][i],data['arrErrMax'][i]))

In [ ]:
import pandas as pd
print('Bảng 1: Classification error của các fixed combining rule')
data_1 = pd.DataFrame.from_dict(dict_data)
data_1

In [ ]:
data['arrErrSum'][0]

# Caculate Win, Equal, Loss

In [ ]:
def win_compare_error(array1, array2):
#     win_result = []
    win_result = 0
    for i in range(len(array1)):
#         win_result.append(1 if array1[i] < array2[i] else 0)
        if (array1[i] < array2[i]):
            win_result += 1
    return win_result

def equal_compare_error(array1, array2):
    equal_result = 0
    for i in range(len(array1)):
        if (array1[i] == array2[i]):
            equal_result += 1
    return equal_result

def loss_compare_error(array1, array2):
    loss_result = 0
    for i in range(len(array1)):
        if (array1[i] > array2[i]):
            loss_result += 1
    return loss_result

In [ ]:
len(data['arrErrSum'][2])

In [ ]:
final_results = []
for i in range(len(cv_filename)):
    # Sum vs Product
    win_compare_sum_prod = win_compare_error(data['arrErrSum'][i],data['arrErrProd'][i])
    print(win_compare_sum_prod)
    equal_compare_sum_prod = equal_compare_error(data['arrErrSum'][i],data['arrErrProd'][i])
    loss_compare_sum_prod = loss_compare_error(data['arrErrSum'][i],data['arrErrProd'][i])
    # Sum vs Min
    win_compare_sum_min = win_compare_error(data['arrErrSum'][i],data['arrErrMin'][i])
    print(win_compare_sum_min)
    equal_compare_sum_min = equal_compare_error(data['arrErrSum'][i],data['arrErrMin'][i])
    loss_compare_sum_min = loss_compare_error(data['arrErrSum'][i],data['arrErrMin'][i])
    # Sum vs Max
    win_compare_sum_max = win_compare_error(data['arrErrSum'][i],data['arrErrMax'][i])
    equal_compare_sum_max = equal_compare_error(data['arrErrSum'][i],data['arrErrMax'][i])
    loss_compare_sum_max = loss_compare_error(data['arrErrSum'][i],data['arrErrMax'][i])
    dict_result = {'Dataset':cv_filename[i],'win_sum_prod':win_compare_sum_prod,'equal_sum_prod':equal_compare_sum_prod,
                  'loss_sum_prod':loss_compare_sum_prod,'win_sum_min':win_compare_sum_min,'equal_sum_min':equal_compare_sum_min,
                  'loss_sum_min':loss_compare_sum_min,'win_sum_max':win_compare_sum_max,'equal_sum_max':equal_compare_sum_max,
                  'loss_sum_max':loss_compare_sum_max}
    print(dict_result)
    final_results.append(dict_result)

In [ ]:
import scipy
#Wilcoxon Sum vs Other Rules
print("DATA: cv_australian.mat")
print("Wilcoxon Sum vs Product Rules:",scipy.stats.wilcoxon(data['arrErrSum'][0],data['arrErrProd'][0],zero_method='wilcox'))
print("Wilcoxon Sum vs Min Rules:",scipy.stats.wilcoxon(data['arrErrSum'][0],data['arrErrMin'][0],zero_method='wilcox'))
print("Wilcoxon Sum vs Max Rules:",scipy.stats.wilcoxon(data['arrErrSum'][0],data['arrErrMax'][0],zero_method='wilcox'))

In [ ]:
print("DATA: cv_bupa.mat")
print("Wilcoxon Sum vs Product Rules:",scipy.stats.wilcoxon(data['arrErrSum'][1],data['arrErrProd'][1],zero_method='wilcox'))
print("Wilcoxon Sum vs Min Rules:",scipy.stats.wilcoxon(data['arrErrSum'][1],data['arrErrMin'][1],zero_method='wilcox'))
print("Wilcoxon Sum vs Max Rules:",scipy.stats.wilcoxon(data['arrErrSum'][1],data['arrErrMax'][1],zero_method='wilcox'))

In [ ]:
print("DATA: cv_glass.mat")
print("Wilcoxon Sum vs Product Rules:",scipy.stats.wilcoxon(data['arrErrSum'][2],data['arrErrProd'][2],zero_method='wilcox'))
print("Wilcoxon Sum vs Min Rules:",scipy.stats.wilcoxon(data['arrErrSum'][2],data['arrErrMin'][2],zero_method='wilcox'))
print("Wilcoxon Sum vs Max Rules:",scipy.stats.wilcoxon(data['arrErrSum'][2],data['arrErrMax'][2],zero_method='wilcox'))

In [ ]:
import pandas as pd
print('Bảng 2: Kết quả kiểm định')
data_2 = pd.DataFrame.from_dict(final_results)
data_2